# Transformer training (chantier en cours)

In [1]:
%load_ext autoreload
%autoreload 2

from transformer_model import *
import nltk
import sys
sys.path.append("../../..")

In [2]:
#import statapp.transformer.common.get_positional_encodings
#from statapp.common.preprocessing import load_data, encode_data, split_into_X_y
def load_data(path, sample=1, split_on=" "):
    """Load a text dataset, optionally sampling the data rounding at some splitting string.
    
    Args:
        path (str): Path to a text file.
        sample (float): (approximate) Proportion of the dataset to load. 1 loads everything.
            Default: 1.
        split_on (str): String to split dataset on, e.g. "\n".
            When sampling, the dataset will be cut at the previous split_on substring.
            Ignored if sample==1.
            If None, samples the dataset by cutting at the previous character.
            Default: " ".
    
    Returns:
        string: Dataset.
    """
    with open(path, "r", encoding="utf-8") as file:
        text = file.read()
    
    if sample != 1:
        text = text[:int(len(text)*sample)]
        if split_on is not None:
            text = text[:-text[::-1].find(split_on)-1]
    
    return text

Preprocessing maison bien moche pour le moment... ^^

In [3]:
text = load_data("data/fr.train.top1M.txt")

In [4]:
tokens = nltk.word_tokenize(text[:1000000])

In [5]:
len(tokens)

193627

In [6]:
vocab = list(set(tokens))

In [7]:
vocab_size = len(vocab)

In [8]:
vocab_numbers = dict(zip(vocab, range(0,len(vocab))))

In [9]:
tokens_numbers = np.array([vocab_numbers[tokens[i]] for i in range(len(tokens))])

In [10]:
tokens_numbers_sequences = np.array([ tokens_numbers[i:i+vector_size] for i in range(len(tokens_numbers)-vector_size)])

In [11]:
tokens_numbers_sequences = torch.tensor(tokens_numbers_sequences , dtype=torch.int64)

In [12]:
nb_sequences =  tokens_numbers_sequences.shape[0]

In [13]:
LMtransformer = Transformer(vocab_size, Decoder(MultiHeadAttention(nb_heads, head_size, vector_size), FeedforwardNetwork(vector_size, ffn_hidden_size)))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(LMtransformer.parameters(), lr=0.001)

In [138]:
loss = nn.CrossEntropyLoss()
input = torch.tensor([[[0.1,0.9],[0.9,0.1],[0.6,0.4]],[[0.6,0.4],[0.1,0.9],[0.6,0.4]]]).reshape(-1,2)
target = torch.tensor([[0,1,0],[1,1,0]]).flatten()
output = loss(input, target)

In [151]:
torch.tensor([[[0.1,0.9],[0.9,0.1],[0.6,0.4]],[[0.6,0.4],[0.1,0.9],[0.6,0.4]]])[:,:-1]

tensor([[[0.1000, 0.9000],
         [0.9000, 0.1000]],

        [[0.6000, 0.4000],
         [0.1000, 0.9000]]])

In [139]:
nn.CrossEntropyLoss()(torch.tensor([[0.9,0.1],[0.6,0.4]]),torch.tensor([0,1]))

tensor(0.5846)

In [140]:
input

tensor([[0.1000, 0.9000],
        [0.9000, 0.1000],
        [0.6000, 0.4000],
        [0.6000, 0.4000],
        [0.1000, 0.9000],
        [0.6000, 0.4000]])

In [141]:
target

tensor([0, 1, 0, 1, 1, 0])

In [142]:
output

tensor(0.7846)

In [154]:
def train_model(nb_epochs, batch_size):
    
    #What is this ??
    LMtransformer.train()
    
    for epoch in range(nb_epochs):
        
        running_loss = 0
        
        randperm = torch.randperm(nb_sequences)
        randperm = randperm[:(nb_sequences//batch_size)*batch_size]
        batchs_indices = randperm.reshape(nb_sequences//batch_size, batch_size)
        
        for batch_indices in batchs_indices:
            
            batch = tokens_numbers_sequences[batch_indices]
            optimizer.zero_grad()
            output = LMtransformer(batch)
            loss = criterion(output[:,:-1].reshape(-1, vocab_size), batch[:,1:].flatten())

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.

In [155]:
torch.autograd.set_detect_anomaly(True)

In [156]:
train_model(1,1)

C:\Users\Eric\statapp_language_model\statapp\transformer\pytorch\transformer_model.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(embedded + pos_encodings, dtype=torch.float32)


tensor([[[-1.9139e+00, -3.5920e+00, -5.7948e-01,  ..., -5.0988e+00,
          -9.9114e-01, -8.0165e-01],
         [-4.0731e-01, -5.2003e-01, -8.5162e-01,  ..., -1.8676e-01,
          -9.6748e-01, -1.3526e+00],
         [-1.1343e+00, -2.5011e-02,  1.4028e-01,  ..., -6.5303e-01,
           8.7634e-01,  1.4608e-03],
         ...,
         [-9.1897e-01,  1.5723e+00,  1.2520e+00,  ..., -1.0496e-01,
           5.2159e-01, -8.6769e-02],
         [-4.6531e-02,  1.1302e+00,  3.1362e-01,  ...,  8.3105e-01,
          -3.7823e-01, -1.2564e+00],
         [ 5.4945e-01,  6.4700e-02, -7.7873e-01,  ...,  4.5154e-01,
           9.3516e-01, -1.2828e+00]]], grad_fn=<AddBackward0>)


C:\Users\Eric\statapp_language_model\statapp\transformer\pytorch\transformer_model.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.finalfc(x))
..\torch\csrc\autograd\python_anomaly_mode.cpp:57: UserWarning: Traceback of forward call that caused the error:
  File "C:\Users\Eric\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Eric\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Eric\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Eric\Anaconda3\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "C:\Users\Eric\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 563, in start
    self.io_loop.start()
  File "C:\Users\Eric\Anaconda3\lib\site-packages\tornado\platform\

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [1, 512, 512]], which is output 0 of AddBackward0, is at version 13; expected version 12 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!